# Preparação do ambiente

## Bibliotecas

In [1]:
import numpy as np
import os.path as osp
import pandas as pd

from joblib import load

## Constantes e funções auxiliares

In [2]:
FILE_SUPERVISAO_MERCADO = '../datasets/supervisao_mercado/supervisao_mercado.xlsx'
FILE_MODEL = '../pretrained_models/clf_marketplaces.joblibj'

In [7]:
class MarketPlacesPredictor():

    def __init__(self, file_model):
        if osp.exists(file_model):
            self.model = load(file_model)
        else:
            self.model = None
            print(f'Model file "{file_model}" not found')

    def predict(self, X):
        if self.model is not None:
            predicted_class = self.model.predict(X)
            predicted_proba = self.model.predict_proba(X)[:,1]
            return np.column_stack((predicted_class,predicted_proba))
        else:
            print('Model is not loaded')
            return None


# Carga e prepação dos dados

In [4]:
dict_df_mercado = pd.read_excel(
    FILE_SUPERVISAO_MERCADO,
    sheet_name=None,
    usecols=[2,8,10],
    names=['texto_busca', 'descricao', 'passivel_homologacao'],
    true_values=['Sim', 'sim'],
    false_values=['Não','não'],
    na_values=['-'])

df_mercado = pd.concat(dict_df_mercado.values())
df_mercado = df_mercado.dropna()
df_mercado['passivel_homologacao'] = df_mercado['passivel_homologacao'].astype(int)
df_mercado

,texto_busca,descricao,passivel_homologacao
0,Smartphone,Positivo Twist 4 Pro S518 Dual SIM 64 GB preto...,1
1,Smartphone,Xiaomi Pocophone Poco X6 Pro 5G Dual SIM 512 G...,1
2,Smartphone,"Smartphone Moto G22 Tela De 6,5 128 Gb 4 Ram P...",1
3,Smartphone,Smartphone Infinix Smart 6 64 Gb Dual Chip 2gb...,1
4,Smartphone,Smartphone Samsung Galaxy A21s Tela 6.5 64gb 4...,1
...,...,...,...
170,Bluetooth Wifi,fone bluetooth sem fio beatfade preto,1
171,Bluetooth Wifi,Fone Bluetooth Sem Fio Wireless Compatível Com...,1
172,Bluetooth Wifi,Fone Bluetooth Sem Fio Preto Caixinha Para Sam...,1
173,Bluetooth Wifi,Fone De Ouvido Bluetooth Sem Fio Tipo Ambie De...,1


# Análise

In [9]:
predictor = MarketPlacesPredictor(FILE_MODEL)
predictor.model

Model file "../pretrained_models/clf_marketplaces.joblibj" not found


In [11]:
X = df_mercado.descricao.values
y = df_mercado.passivel_homologacao.values

prediction = predictor.predict(X)

if prediction is not None:
    predicted_class = prediction[:,0]
    predicted_proba = prediction[:,1]
    
    df_mercado['classe_predita'] = predicted_class.astype(int)
    df_mercado['probabilidade'] = predicted_proba
    
    print('Accuracy of model on validation set: {:.3f}'
         .format(predictor.model.score(X, y)))
    
    df_mercado.pivot_table(index='passivel_homologacao',columns='classe_predita',values='descricao',aggfunc='count')

Model is not loaded


In [ ]:
predictor.predict(['Repetidor De Sinal Internet 600mbps Branco 110v/220v Wifi'])